In [28]:
import requests
from bs4 import BeautifulSoup
import re

In [61]:
# Function to remove tags
def remove_tags(html):

    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()

    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)


In [ ]:
for i in range(1,1291):
    if (i % 10) == 0:
        print(i)

    url = f"https://api.digitale-sammlungen.de/ocr/bsb10589853/{i}"
    ocr_html = requests.get(url).text
    ocr_converted = ocr_html.encode('iso-8859-1').decode('utf-8')

    ocr_text = remove_tags(ocr_converted)
    text = re.sub('bsb10589853_[0-9]{5} ', '', ocr_text)
    text = re.sub(r' [\/,.:!;=\(\)\-](?= )', '', text)
    text = re.sub(r'ſ', 's', text)
    #text = re.sub(r' [0-9]+.?(?= )', '', text)
    text = re.sub(r'\s+', ' ', text)

    with open(f"source_texts/praxis_pietatis_melica/{i}.txt", "w") as f:
        f.write(text)